In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score
import gensim.models

In [17]:
df = pd.read_csv('data/data/usa_2015/ticker_data_preprocessed.csv', index_col=0)
print(df.shape)
df.head()

(457, 1201)


,2015-03-27 00:00:00,2015-03-30 00:00:00,2015-03-31 00:00:00,2015-04-01 00:00:00,2015-04-02 00:00:00,2015-04-06 00:00:00,2015-04-07 00:00:00,2015-04-08 00:00:00,2015-04-09 00:00:00,2015-04-10 00:00:00,...,2019-12-18 00:00:00,2019-12-19 00:00:00,2019-12-20 00:00:00,2019-12-23 00:00:00,2019-12-24 00:00:00,2019-12-26 00:00:00,2019-12-27 00:00:00,2019-12-30 00:00:00,2019-12-31 00:00:00,sector
A,0.010074,0.014838,-0.004075,-0.003851,0.015946,-0.002854,0.012163,-0.004241,0.005443,0.024947,...,-0.006194,0.012945,0.008046,0.000822,0.000469,0.001758,-0.000351,-0.006088,0.004829,Healthcare
AAL,0.027111,0.018610,-0.016033,-0.044335,-0.024980,-0.022367,-0.012063,0.026105,-0.018465,-0.002299,...,-0.009901,0.019643,0.017163,0.008953,-0.004096,0.016792,-0.041456,-0.004923,0.013428,Industrials
AAP,0.013252,0.004558,-0.015715,-0.039682,0.001600,0.012154,0.006725,0.014314,-0.005107,0.014522,...,0.000317,-0.009808,0.010481,0.011700,0.003751,-0.008096,-0.005776,0.004863,0.006536,Consumer Cyclical
AAPL,-0.007968,0.025314,-0.015352,-0.001447,0.008612,0.016199,-0.010522,-0.003254,0.007643,0.004267,...,-0.002389,0.001001,-0.002071,0.016318,0.000951,0.019840,-0.000379,0.005935,0.007307,Technology
ABBV,0.006635,0.010234,0.005151,-0.024599,-0.001576,0.010525,0.014060,0.006676,0.025676,0.027851,...,-0.008326,-0.006269,0.005858,0.010751,-0.004432,-0.000223,-0.007013,-0.007623,0.000226,Healthcare


In [18]:
df_pct = df.drop(['sector'], axis=1).T
df_pct.index = pd.to_datetime(df_pct.index)

tickers_list = df_pct.columns.tolist()

df_pct.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2015-03-27,0.010074,0.027111,0.013252,-0.007968,0.006635,0.009814,0.018543,0.006021,-0.002442,0.024730,...,-0.004808,0.002624,-0.008776,0.005344,0.008653,-0.003055,0.015997,0.011807,-0.011597,0.002386
2015-03-30,0.014838,0.018610,0.004558,0.025314,0.010234,0.002650,0.044944,0.006413,0.005110,0.101462,...,0.009584,0.010177,0.024527,0.009648,0.010866,0.014047,0.009106,0.006979,0.021953,0.015145
2015-03-31,-0.004075,-0.016033,-0.015715,-0.015352,0.005151,0.001586,-0.025725,-0.015930,-0.007731,-0.027928,...,-0.012396,0.002015,-0.007357,-0.007605,-0.009335,-0.008689,-0.008856,0.030675,0.000000,-0.013427
2015-04-01,-0.003851,-0.044335,-0.039682,-0.001447,-0.024599,-0.014428,-0.023191,-0.009497,-0.001388,-0.006825,...,-0.002780,0.001724,-0.006353,-0.012380,0.011136,0.001524,-0.011317,-0.012787,-0.002593,-0.000432
2015-04-02,0.015946,-0.024980,0.001600,0.008612,-0.001576,0.003213,0.011442,0.005230,0.006627,-0.005274,...,0.030829,-0.003155,-0.001894,-0.000597,0.010731,0.012050,0.006283,-0.002456,0.006684,0.004971


In [7]:
scores = []
X = df_pct.values.reshape(-1,1)

In [48]:
X.shape

(520260, 1)

In [49]:
model = KMeans(n_clusters=200)
model.fit(X)
labels = model.predict(X)

In [50]:
X.shape, labels.shape

((520260, 1), (520260,))

In [51]:
mapping = {x:token for x, token in zip(X.flatten(), labels)}

In [52]:
df_pct_tok = df_pct.copy()
for col in df_pct:
    df_pct_tok[col] = df_pct[col].map(mapping)

In [53]:
df_pct_tok.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADI,ADM,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
2012-04-02,196,18,135,73,86,167,0,162,86,75,...,121,135,104,104,196,75,39,1,36,92
2012-04-03,131,136,26,51,4,84,92,149,154,32,...,64,183,119,64,9,88,40,38,161,104
2012-04-04,35,162,174,17,113,2,76,17,62,192,...,164,18,9,62,160,160,179,149,64,96
2012-04-05,147,159,76,1,53,88,88,76,199,127,...,193,163,63,38,191,36,167,95,63,117
2012-04-09,152,61,193,104,166,171,113,160,72,18,...,14,120,127,32,4,14,62,84,64,129


In [54]:
sentences = df_pct_tok.values.tolist()

In [55]:
model = gensim.models.Word2Vec(sentences, min_count=1)

In [56]:
embeddings = pd.DataFrame()
for col in df_pct_tok:
    embedding = np.zeros((100,))
    for i in df_pct_tok[col]:
        embedding += model.wv.get_vector(i)
    embedding /= df_pct_tok.shape[0]
    embeddings = pd.concat((embeddings, pd.Series(embedding)), axis=1)

In [57]:
embeddings

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0.027565,0.084233,0.014973,0.027445,-0.011365,0.069233,-0.001709,-0.001617,0.013993,0.017029,...,0.018692,0.067669,-0.014854,-0.007957,0.000877,0.016008,0.011564,0.001474,0.031436,0.037232
1,0.032141,0.090952,0.016714,0.028802,0.012772,0.082311,0.010313,0.010974,0.021461,0.031093,...,0.024663,0.038703,0.007866,-0.008360,0.007297,0.023629,0.020552,0.015740,0.041307,0.056158
2,-0.113984,-0.134170,-0.109942,-0.115079,-0.094903,-0.126197,-0.101154,-0.101488,-0.108481,-0.109168,...,-0.110977,-0.131435,-0.096284,-0.100731,-0.104433,-0.109617,-0.107385,-0.102957,-0.113574,-0.115417
3,0.031708,-0.113313,0.066647,0.035482,0.109167,-0.081654,0.100620,0.097602,0.062638,0.048521,...,0.052019,-0.039281,0.121535,0.127470,0.098966,0.057521,0.067348,0.088661,0.015696,-0.009640
4,-0.008769,0.159147,-0.047586,-0.009883,-0.124400,0.114559,-0.098368,-0.097481,-0.048238,-0.038456,...,-0.034443,0.111025,-0.135156,-0.117935,-0.090256,-0.043577,-0.054912,-0.088452,0.002011,0.025617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.100569,0.113437,0.096698,0.100198,0.091225,0.109727,0.091986,0.093340,0.095821,0.097327,...,0.097520,0.105693,0.089911,0.089065,0.092855,0.097166,0.095844,0.093239,0.100680,0.103216
96,0.050965,0.063677,0.048493,0.050375,0.053507,0.064546,0.049894,0.049951,0.051743,0.055065,...,0.050698,0.046646,0.050918,0.044108,0.046786,0.051179,0.050617,0.051725,0.056064,0.060135
97,-0.018459,-0.073064,-0.005612,-0.018749,0.024780,-0.056997,0.012573,0.012153,-0.003715,-0.005343,...,-0.008937,-0.062743,0.027295,0.016712,0.008827,-0.005623,-0.001805,0.010002,-0.020384,-0.024860
98,-0.137874,-0.048097,-0.159432,-0.140530,-0.178695,-0.063997,-0.176904,-0.175790,-0.155171,-0.144157,...,-0.150049,-0.102599,-0.187061,-0.196785,-0.178229,-0.152435,-0.157686,-0.169551,-0.125779,-0.107933


In [58]:
embeddings.T.to_csv("signal2vec_embds2012.csv", index=None)

In [19]:
a = pd.read_csv('signal2vec_embds2015.csv')
a.index = df_pct.columns

In [20]:
a.to_csv("signal2vec_embds.csv", index=None)